# 3. Implementing a Network from Scratch

A neural network is just layers connected together!
Output of one layer becomes input to the next layer.
Let's build a complete network!


In [ ]:
import torch
import numpy as np


## 1. What is a Neural Network?

A neural network is layers stacked together!
Input → Layer 1 → Layer 2 → ... → Output
Each layer transforms the data into a new representation.


In [ ]:
# Reuse Layer class from previous lesson
class Layer:
    """Layer with multiple neurons"""
    
    def __init__(self, num_inputs, num_neurons, activation='sigmoid'):
        self.weights = torch.randn(num_neurons, num_inputs) * 0.1
        self.biases = torch.randn(num_neurons) * 0.1
        self.activation_name = activation
    
    def activate(self, x):
        if self.activation_name == 'sigmoid':
            return torch.sigmoid(x)
        elif self.activation_name == 'relu':
            return torch.relu(x)
        elif self.activation_name == 'tanh':
            return torch.tanh(x)
        else:
            return x
    
    def forward(self, inputs):
        if inputs.dim() == 1:
            weighted_sum = inputs @ self.weights.T + self.biases
        else:
            weighted_sum = inputs @ self.weights.T + self.biases
        return self.activate(weighted_sum)

# Manual network: 2 layers connected
# Input: 2 features
# Layer 1: 3 neurons (hidden layer)
# Layer 2: 1 neuron (output layer)

inputs = torch.tensor([1.0, 2.0])

print("Neural Network (2 layers):")
print(f"Input: {inputs}")
print()

# Layer 1
layer1 = Layer(num_inputs=2, num_neurons=3, activation='sigmoid')
hidden_output = layer1.forward(inputs)
print(f"Layer 1 output (3 neurons): {hidden_output}")
print()

# Layer 2 (takes Layer 1 output as input)
layer2 = Layer(num_inputs=3, num_neurons=1, activation='sigmoid')
final_output = layer2.forward(hidden_output)
print(f"Layer 2 output (1 neuron): {final_output}")
print()

print("Network flow:")
print(f"Input (2) → Layer 1 (3) → Layer 2 (1) → Output")
print(f"          {inputs.shape} → {hidden_output.shape} → {final_output.shape}")


## 2. Neural Network Class from Scratch

Let's create a network class that contains multiple layers!


In [ ]:
class NeuralNetwork:
    """Neural network from scratch"""
    
    def __init__(self, layer_sizes, activations=None):
        """
        layer_sizes: list of layer sizes [input_size, hidden1, hidden2, ..., output_size]
        activations: list of activation functions for each layer
        """
        self.layers = []
        
        # Default to sigmoid for all layers
        if activations is None:
            activations = ['sigmoid'] * (len(layer_sizes) - 1)
        
        # Create layers
        for i in range(len(layer_sizes) - 1):
            num_inputs = layer_sizes[i]
            num_neurons = layer_sizes[i + 1]
            activation = activations[i]
            
            layer = Layer(num_inputs, num_neurons, activation)
            self.layers.append(layer)
    
    def forward(self, inputs):
        """Forward pass through all layers"""
        x = inputs
        
        # Pass through each layer sequentially
        for i, layer in enumerate(self.layers):
            x = layer.forward(x)
            # Optional: print intermediate outputs
            # print(f"Layer {i+1} output shape: {x.shape}")
        
        return x

# Create a network: 2 inputs → 3 neurons → 2 neurons → 1 output
network = NeuralNetwork(
    layer_sizes=[2, 3, 2, 1],
    activations=['sigmoid', 'sigmoid', 'sigmoid']
)

print("Neural Network Architecture:")
print("Input: 2 features")
print("Layer 1: 3 neurons (hidden layer 1)")
print("Layer 2: 2 neurons (hidden layer 2)")
print("Layer 3: 1 neuron (output layer)")
print()

# Test the network
inputs = torch.tensor([1.0, 2.0])
output = network.forward(inputs)

print(f"Input: {inputs}")
print(f"Output: {output}")
print()

# Show intermediate outputs
print("Forward pass through network:")
x = inputs
for i, layer in enumerate(network.layers):
    x = layer.forward(x)
    print(f"After Layer {i+1}: {x} (shape: {x.shape})")


In [ ]:
# Network 1: Simple network (2 → 4 → 1)
network1 = NeuralNetwork([2, 4, 1], ['sigmoid', 'sigmoid'])
print("Network 1: 2 → 4 → 1")
print(f"Layers: {len(network1.layers)}")
print(f"Total neurons: 4 + 1 = 5")
print()

# Network 2: Deeper network (3 → 5 → 3 → 1)
network2 = NeuralNetwork([3, 5, 3, 1], ['sigmoid', 'sigmoid', 'sigmoid'])
print("Network 2: 3 → 5 → 3 → 1")
print(f"Layers: {len(network2.layers)}")
print(f"Total neurons: 5 + 3 + 1 = 9")
print()

# Network 3: Wide network (2 → 10 → 1)
network3 = NeuralNetwork([2, 10, 1], ['relu', 'sigmoid'])
print("Network 3: 2 → 10 → 1 (ReLU hidden, sigmoid output)")
print(f"Layers: {len(network3.layers)}")
print(f"Total neurons: 10 + 1 = 11")
print()

# Test all networks with same input
test_input = torch.tensor([1.0, 2.0])

print("Testing networks with input = [1.0, 2.0]:")
print(f"Network 1 output: {network1.forward(test_input):.4f}")
print(f"Network 2 output: {network2.forward(test_input[:2]):.4f}")  # First 2 elements
print(f"Network 3 output: {network3.forward(test_input):.4f}")

# Count total parameters
def count_parameters(network):
    total = 0
    for layer in network.layers:
        total += layer.weights.numel() + layer.biases.numel()
    return total

print("\nTotal parameters (weights + biases):")
print(f"Network 1: {count_parameters(network1)}")
print(f"Network 2: {count_parameters(network2)}")
print(f"Network 3: {count_parameters(network3)}")


## 4. Batch Processing with Networks

Networks can process batches of inputs efficiently!


In [ ]:
# Create a network
network = NeuralNetwork([2, 4, 1], ['sigmoid', 'sigmoid'])

# Single input
single_input = torch.tensor([1.0, 2.0])
single_output = network.forward(single_input)

print("Single input:")
print(f"Input: {single_input}")
print(f"Output: {single_output}")
print()

# Batch of inputs (4 samples)
batch_inputs = torch.tensor([[1.0, 2.0],
                             [3.0, 4.0],
                             [5.0, 6.0],
                             [7.0, 8.0]])

batch_outputs = network.forward(batch_inputs)

print("Batch of inputs:")
print(f"Input shape: {batch_inputs.shape} (batch_size=4, features=2)")
print(f"Input:\n{batch_inputs}")
print()
print(f"Output shape: {batch_outputs.shape} (batch_size=4, outputs=1)")
print(f"Outputs:\n{batch_outputs}")
print()

print("Notice: Network processes entire batch at once!")
print("This is much faster than processing one at a time.")


## 5. Key Takeaways

**What a neural network is:**
- Multiple layers connected together
- Output of one layer = input to next layer
- Transforms inputs through multiple representations

**Network architecture:**
- Input layer: receives data
- Hidden layers: transform data (can have many!)
- Output layer: produces final prediction

**Key concepts:**
- **Depth**: Number of layers (more layers = deeper network)
- **Width**: Number of neurons per layer (more neurons = wider network)
- **Forward pass**: Data flows from input → output

**Forward pass:**
1. Input goes to first layer
2. Each layer's output becomes next layer's input
3. Final layer produces network output

**Remember:** Networks learn complex patterns by stacking simple transformations!
